In [3]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from clickhouse_connect import get_client

load_dotenv()

True

In [21]:
IdGuten_Ozon = os.getenv('IdGuten_Ozon')
KeyGuten_Ozon = os.getenv('KeyGuten_Ozon')

In [34]:

def get_realization_by_day(client_id, api_key, date_params):
    url = "https://api-seller.ozon.ru/v1/finance/realization/by-day"

    headers = {
        "Client-Id": client_id,
        "Api-Key": api_key,
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, data=json.dumps(date_params))

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")  # Print error status code
        print(f"Message: {response.text}")        # Print error message
        return {
            "error": f"Failed to retrieve data: {response.status_code}",
            "message": response.text
        }

def save_result_to_file(result, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=2, ensure_ascii=False)

# Function to flatten the JSON structure
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Example usage
client_id = IdGuten_Ozon
api_key = KeyGuten_Ozon

# Get yesterday's date
end_date = datetime.now() - timedelta(days=1)

# Initialize an empty list to store all results
all_results = []

# Loop over the last 31 days starting from yesterday
for i in range(31):
    # Calculate the date for the current iteration
    current_date = end_date - timedelta(days=i)
    day = current_date.day
    month = current_date.month
    year = current_date.year

    # Construct the date parameters as a JSON object with integer values
    date_params = {
        "day": day,
        "month": month,
        "year": year
    }

    # Get the realization data for the current date
    result = get_realization_by_day(client_id, api_key, date_params)

    # Debug: Print the result to check its structure
    print(f"Result for {current_date.strftime('%Y-%m-%d')}: {result}")

    # Check if the result contains data
    if 'rows' in result:
        # Flatten the JSON data and add date information
        flat_data = [flatten_json(item) for item in result['rows']]
        for item in flat_data:
            item['date'] = f"{year}-{month}-{day}"  # Ensure date is added correctly

        # Append the flattened data to the list of all results
        all_results.extend(flat_data)
    else:
        print(f"No data for {current_date.strftime('%Y-%m-%d')}")

# Create a DataFrame from all results
df = pd.DataFrame(all_results)

# Save the result to a text file
output_filename = "realization_report_last_31_days.txt"
save_result_to_file(all_results, output_filename)

print(f"Result saved to {output_filename}")

Result for 2025-04-10: {'rows': [{'item': {'name': 'Ирригатор Braun Oral-B Professional Care OxyJet MD20', 'offer_id': 'Б0005270', 'barcode': '4210201442028', 'sku': 148765553}, 'delivery_commission': None, 'return_commission': {'price_per_instance': 8917, 'quantity': 1, 'amount': 8917, 'compensation': 0, 'commission': 0, 'bonus': 2140.71, 'standard_fee': 1974.46, 'total': 9105.54, 'stars': 0, 'bank_coinvestment': 22.29, 'pick_up_point_coinvestment': 0}, 'commission_ratio': 0.18, 'rowNumber': 1, 'seller_price_per_instance': 11080}, {'item': {'name': 'Сетка и режущий блок Braun Series 3 21B', 'offer_id': '81570020', 'barcode': '4210201163596', 'sku': 154813850}, 'delivery_commission': {'price_per_instance': 2557, 'quantity': 1, 'amount': 2557, 'compensation': 0, 'commission': 0, 'bonus': 788.61, 'standard_fee': 804.48, 'total': 2547.52, 'stars': 0, 'bank_coinvestment': 6.39, 'pick_up_point_coinvestment': 0}, 'return_commission': None, 'commission_ratio': 0.24, 'rowNumber': 2, 'seller_pr

In [36]:
date = df["date"]
date

0       2025-4-10
1       2025-4-10
2       2025-4-10
3       2025-4-10
4       2025-4-10
          ...    
3329    2025-3-11
3330    2025-3-11
3331    2025-3-11
3332    2025-3-11
3333    2025-3-11
Name: date, Length: 3334, dtype: object